# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096854


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:  10%|▉         | 3/31 [00:07<01:06,  2.38s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<00:45,  1.70s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:31,  1.23s/it]

Rendering models:  19%|█▉        | 6/31 [00:07<00:22,  1.12it/s]

...

Rendering models:  84%|████████▍ | 26/31 [00:10<00:00,  6.08it/s]

Rendering models:  90%|█████████ | 28/31 [00:10<00:00,  7.11it/s]

Rendering models:  94%|█████████▎| 29/31 [00:10<00:00,  6.81it/s]

Rendering models:  97%|█████████▋| 30/31 [00:10<00:00,  7.45it/s]

Alexander00                           0.000763
Idax98                                0.000373
jnarayanbvg                           0.012512
Jonas_Cross                           0.000439
wilderbeast2                          0.004673
Quarkgc                               0.000216
fortnitemarter                        0.014622
not-logged-in-c6917d9b19f41e6c2995    0.004724
babyduckyt                            0.006645
insert_name                           0.019690
not-logged-in-ea8ed61a7274cfb6a43b    0.018934
ericmalani                            0.000722
not-logged-in-b29906a36b4ade3fcb65    0.003364
tominas                               0.000428
acapirala                             0.000217
not-logged-in-300dca7f45d1a5d1dd58    0.027155
Lavadude                              0.014799
layne.woodward2701                    0.024430
equidad1                              0.000179
Tessaje04                             0.684580
not-logged-in-87670b37bb858efe958e    0.000211
jzeiszler    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())